# GitHub Group Creation

> Create GitHub group based on the provided group information

In [ ]:
#| default_exp gh_group

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
from pprint import pprint

In [ ]:
#| export
from github import Github
import github
import json
import time

In [ ]:
#| export
#| hide
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [ ]:
#| export
class GitHubGroup:
    def __init__(self,
                 credentials_fp="", # the file path to the credential json
                 org="", # the organization name
                 verbosity=0 # Controls the verbosity: 0=slient, 1=print status
                ):
        self.github = None
        self.org = None
        self.verbosity = verbosity
        
        if credentials_fp != "":
            self.auth_github(credentials_fp)
        if org != "":
            self.set_org(org)

    def auth_github(self,
                    credentials_fp: str # the personal access token generated at GitHub Settings
                   ):
        "Authenticate GitHub account with the provided credentials"
        with open(credentials_fp, "r") as f:
            token = json.load(f)["GitHub Token"]
        self.github = Github(token)
        # check authorization
        _ = self.github.get_user().get_repos()[0]
        if self.verbosity != 0:
            print(f"Successfully Authenticated. "
                  f"GitHub account: {bcolors.OKGREEN} {self.github.get_user().login} {bcolors.ENDC}")
        
    def set_org(self,
                org: str # the target organization name
               ):
        "Set the target organization for repo creation"
        self.org = self.github.get_organization(org)
        if self.verbosity != 0:
            print(f"Target Organization Set: {bcolors.OKGREEN} {self.org.login} {bcolors.ENDC}")

    def create_repo(self,
                    repo_name: str, # repository name
                    repo_template="", # template repository that new repo will use. If empty string, an empty repo will be created. Put in the format of "<owner>/<repo>"
                    private=True, # visibility of the created repository
                    description="", # description for the GitHub repository
                    personal_account=False, # create repos in personal GitHub account
                    ) -> github.Repository.Repository:
        "Create a repository, either blank, or from a template"
        if self.org is None and personal_account:
            raise ValueError("Organization is not set")
        if personal_account:
            parent = self.github.get_user()
        else:
            parent = self.org
        if repo_template == "":
            return parent.create_repo(
                name=repo_name,
                private=private,
                description=description
            )
        # create from template
        return parent.create_repo_from_template(
            name=repo_name,
            repo=self.get_repo(repo_template),
            private=private,
            description=description,
        )
    
    def get_repo(self,
                 repo_full_name: str # full name of the target repository
                ) -> github.Repository.Repository:
        "To get a repository by its name"
        return self.github.get_repo(repo_full_name)
    
    def get_org_repo(self,
                     repo_full_name: str # full name of the target repository
                    ) -> github.Repository.Repository:
        "Get a repository within the target organization"
        return self.org.get_repo(repo_full_name)

    
    def get_team(self,
                 team_slug:str # team slug of the team
                ) -> github.Team.Team:
        "Get the team inside the target organization"
        if self.org is None:
            raise ValueError("The organization has not been set. Please set it via g.set_org")
        return self.org.get_team_by_slug(team_slug)
    
    def rename_files(self,
                     repo: github.Repository.Repository, # the repository that we want to rename file
                     og_filename: str, # old file name
                     new_filename: str # new file name
                    ):
        "Rename the file by delete the old file and commit the new file"
        file = repo.get_contents(og_filename)
        repo.create_file(new_filename, "rename files", file.decoded_content)
        repo.delete_file(og_filename, "delete old files", file.sha)
        if self.verbosity != 0:
            print(f"File Successfully Renamed from "
                  f" {bcolors.OKCYAN} {og_filename} {bcolors.ENDC} "
                  f" to {bcolors.OKGREEN} {new_filename} {bcolors.ENDC}")
        
    def add_collaborator(self,
                          repo: github.Repository.Repository, # target repository
                          collaborator:str, # GitHub username of the collaborator
                          permission:str # `pull`, `push` or `admin`
                         ):
        "Add collaborator to the repository with specified permission"
        repo.add_to_collaborators(collaborator, permission)
        if self.verbosity != 0:
            print(f"Added Collaborator: {bcolors.OKGREEN} {collaborator} {bcolors.ENDC}"
                  f" to: {bcolors.OKGREEN} {repo.name} {bcolors.ENDC} with"
                  f"permission: {bcolors.OKGREEN} {permission} {bcolors.ENDC}")
        
    def add_team(self,
                  repo: github.Repository.Repository, # target repository
                  team: github.Team.Team, # GitHub username of the collaborator
                  permission:str # `pull`, `push` or `admin`
                 ):
        "Add team to the repository with specified permission"
        team.add_to_repos(repo)
        team.update_team_repository(repo, permission)
        if self.verbosity != 0:
            print(f"Team {bcolors.OKGREEN} {team.name} {bcolors.ENDC} "
                  f"added to {bcolors.OKGREEN} {repo.name} {bcolors.ENDC} "
                  f"with permission {bcolors.OKGREEN} {permission} {bcolors.ENDC}")
        
    def create_group_repo(self,
                          repo_name: str, # group repository name
                          collaborators: [str], # list of collaborators GitHub id
                          permission: str, # the permission of collaborators. `pull`, `push` or `admin`
                          rename_files=dict(), # dictionary of files renames {<og_name>:<new_name>}
                          repo_template="", # If empty string, an empty repo will be created. Put in the format of "<owner>/<repo>"
                          private=True, # visibility of the created repository
                          description="", # description for the GitHub repository
                          team_slug="", # team slug, add to this repo
                          team_permission="", # team permission to this repository `pull`, `push` or `admin`
                         ) -> github.Repository.Repository: # created repository
        "Create a Group Repository"
        repo = self.create_repo(repo_name, repo_template, private, description)
        if self.verbosity != 0:
            print(f"Repo {bcolors.OKGREEN} {repo.name} {bcolors.ENDC} Created... Wait for 3 sec to updates")
        time.sleep(3)
        for og_name, new_name in rename_files.items():
            self.rename_files(repo, og_name, new_name)
        for collaborator in collaborators:
            self.add_collaborator(repo, collaborator, permission)
        if team_slug != "":
            g_team = self.get_team(team_slug)
            self.add_team(repo, g_team, team_permission)
        if self.verbosity != 0:
            print(f"Group Repo: {bcolors.OKGREEN} {repo_name} {bcolors.ENDC} successfuly created!")
        return repo

In [ ]:
#| export
class GitHubGroup:
    def __init__(self,
                 credentials_fp="", # the file path to the credential json
                 org="", # the organization name
                 verbosity=0 # Controls the verbosity: 0=slient, 1=print status
                ):
        self.github = None
        self.org = None
        self.verbosity = verbosity
        
        if credentials_fp != "":
            self.auth_github(credentials_fp)
        if org != "":
            self.set_org(org)

    def auth_github(self,
                    credentials_fp: str # the personal access token generated at GitHub Settings
                   ):
        "Authenticate GitHub account with the provided credentials"
        with open(credentials_fp, "r") as f:
            token = json.load(f)["GitHub Token"]
        self.github = Github(token)
        # check authorization
        _ = self.github.get_user().get_repos()[0]
        if self.verbosity != 0:
            print(f"Successfully Authenticated. "
                  f"GitHub account: {bcolors.OKGREEN} {self.github.get_user().login} {bcolors.ENDC}")
        
    def set_org(self,
                org: str # the target organization name
               ):
        "Set the target organization for repo creation"
        self.org = self.github.get_organization(org)
        if self.verbosity != 0:
            print(f"Target Organization Set: {bcolors.OKGREEN} {self.org.login} {bcolors.ENDC}")

    def create_repo(self,
                    repo_name: str, # repository name
                    repo_template="", # template repository that new repo will use. If empty string, an empty repo will be created. Put in the format of "<owner>/<repo>"
                    private=True, # visibility of the created repository
                    description="", # description for the GitHub repository
                    personal_account=False, # create repos in personal GitHub account
                    ) -> github.Repository.Repository:
        "Create a repository, either blank, or from a template"
        if self.org is None and personal_account:
            raise ValueError("Organization is not set")
        if personal_account:
            parent = self.github.get_user()
        else:
            parent = self.org
        if repo_template == "":
            return parent.create_repo(
                name=repo_name,
                private=private,
                description=description
            )
        # create from template
        return parent.create_repo_from_template(
            name=repo_name,
            repo=self.get_repo(repo_template),
            private=private,
            description=description,
        )
    
    def get_repo(self,
                 repo_full_name: str # full name of the target repository
                ) -> github.Repository.Repository:
        "To get a repository by its name"
        return self.github.get_repo(repo_full_name)
    
    def get_org_repo(self,
                     repo_full_name: str # full name of the target repository
                    ) -> github.Repository.Repository:
        "Get a repository within the target organization"
        return self.org.get_repo(repo_full_name)

    
    def get_team(self,
                 team_slug:str # team slug of the team
                ) -> github.Team.Team:
        "Get the team inside the target organization"
        if self.org is None:
            raise ValueError("The organization has not been set. Please set it via g.set_org")
        return self.org.get_team_by_slug(team_slug)
    
    def rename_files(self,
                     repo: github.Repository.Repository, # the repository that we want to rename file
                     og_filename: str, # old file name
                     new_filename: str # new file name
                    ):
        "Rename the file by delete the old file and commit the new file"
        file = repo.get_contents(og_filename)
        repo.create_file(new_filename, "rename files", file.decoded_content)
        repo.delete_file(og_filename, "delete old files", file.sha)
        if self.verbosity != 0:
            print(f"File Successfully Renamed from "
                  f" {bcolors.OKCYAN} {og_filename} {bcolors.ENDC} "
                  f" to {bcolors.OKGREEN} {new_filename} {bcolors.ENDC}")
        
    def add_collaborator(self,
                          repo: github.Repository.Repository, # target repository
                          collaborator:str, # GitHub username of the collaborator
                          permission:str # `pull`, `push` or `admin`
                         ):
        "Add collaborator to the repository with specified permission"
        repo.add_to_collaborators(collaborator, permission)
        if self.verbosity != 0:
            print(f"Added Collaborator: {bcolors.OKGREEN} {collaborator} {bcolors.ENDC}"
                  f" to: {bcolors.OKGREEN} {repo.name} {bcolors.ENDC} with "
                  f"permission: {bcolors.OKGREEN} {permission} {bcolors.ENDC}")
    
    def remove_collaborator(self,
                            repo: github.Repository.Repository, # target repository
                            collaborator:str, # GitHub username of the collaborator
                           ):
        "Remove collaborator privilages from the repository"
        repo.remove_from_collaborators(collaborator)
        
    def resend_invitation(self,
                          repo: github.Repository.Repository, # target repository
                         ) -> [github.NamedUser.NamedUser]: # list of re-invited user
        "Resent Invitation to invitee who did not accept the invitation"
        ...
        
    def add_team(self,
                  repo: github.Repository.Repository, # target repository
                  team: github.Team.Team, # GitHub username of the collaborator
                  permission:str # `pull`, `push` or `admin`
                 ):
        "Add team to the repository with specified permission"
        team.add_to_repos(repo)
        team.update_team_repository(repo, permission)
        if self.verbosity != 0:
            print(f"Team {bcolors.OKGREEN} {team.name} {bcolors.ENDC} "
                  f"added to {bcolors.OKGREEN} {repo.name} {bcolors.ENDC} "
                  f"with permission {bcolors.OKGREEN} {permission} {bcolors.ENDC}")
        
    def create_group_repo(self,
                          repo_name: str, # group repository name
                          collaborators: [str], # list of collaborators GitHub id
                          permission: str, # the permission of collaborators. `pull`, `push` or `admin`
                          rename_files=dict(), # dictionary of files renames {<og_name>:<new_name>}
                          repo_template="", # If empty string, an empty repo will be created. Put in the format of "<owner>/<repo>"
                          private=True, # visibility of the created repository
                          description="", # description for the GitHub repository
                          team_slug="", # team slug, add to this repo
                          team_permission="", # team permission to this repository `pull`, `push` or `admin`
                         ) -> github.Repository.Repository: # created repository
        "Create a Group Repository"
        repo = self.create_repo(repo_name, repo_template, private, description)
        if self.verbosity != 0:
            print(f"Repo {bcolors.OKGREEN} {repo.name} {bcolors.ENDC} Created... Wait for 3 sec to updates")
        time.sleep(3)
        for og_name, new_name in rename_files.items():
            self.rename_files(repo, og_name, new_name)
        for collaborator in collaborators:
            self.add_collaborator(repo, collaborator, permission)
        if team_slug != "":
            g_team = self.get_team(team_slug)
            self.add_team(repo, g_team, team_permission)
        if self.verbosity != 0:
            print(f"Group Repo: {bcolors.OKGREEN} {repo_name} {bcolors.ENDC} successfuly created!")
        return repo


# GitHub Authentication

View this [document](https://docs.google.com/document/d/1RvZnOX6nh0bXn6Zh4U-Yxazukuez5oGrtcP8mN-Roco/edit?usp=sharing) for how to set up your GitHub Personal Access Token. (TODO: be sure to specify scopes)

Store the token information in a json file.

In [ ]:
credentials_fp = "credentials.json"
with open(credentials_fp, "r") as f:
    # take a look at the token
    print(f.read())
credentials_fp = "../../credentials.json" #| hide_line
g = GitHubGroup(credentials_fp=credentials_fp, org="COGS118A")

{
  "GitHub Token": "token",
  "Canvas Token": "token"
}


In [ ]:
credentials_fp = "credentials.json"
with open(credentials_fp, "r") as f:
    # take a look at the token
    print(f.read())
credentials_fp = "../../credentials.json" #| hide_line
g = GitHubGroup(credentials_fp=credentials_fp, org="COGS118A", verbosity=1)

{
  "GitHub Token": "token",
  "Canvas Token": "token"
}
Successfully Authenticated. GitHub account:  scott-yj-yang 
Target Organization Set:  COGS118A 


Optionally, you can instansiate a GitHubGroup object and authenticate yourself by calling the following method.

In [ ]:
show_doc(GitHubGroup.auth_github)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/gh_group.py#L192){target="_blank" style="float:right; font-size:smaller"}

### GitHubGroup.auth_github

>      GitHubGroup.auth_github (credentials_fp:str)

Authenticate GitHub account with the provided credentials

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| credentials_fp | str | the personal access token generated at GitHub Settings |

In [ ]:
g = GitHubGroup(verbosity=1) # you can set verbosity to 1 to see the current progress
g.auth_github(credentials_fp)

Successfully Authenticated. GitHub account:  scott-yj-yang 


# GitHub Organization Settings

Usually, you want to create students repositories under a course GitHub organization. To set the target organization, you can call the following function.

In [ ]:
show_doc(GitHubGroup.set_org)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/gh_group.py#L205){target="_blank" style="float:right; font-size:smaller"}

### GitHubGroup.set_org

>      GitHubGroup.set_org (org:str)

Set the target organization for repo creation

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| org | str | the target organization name |

In [ ]:
g.set_org("COGS118A")

Target Organization Set:  COGS118A 


# Create GitHub Group in one Call

In [ ]:
show_doc(GitHubGroup.create_group_repo)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/gh_group.py#L314){target="_blank" style="float:right; font-size:smaller"}

### GitHubGroup.create_group_repo

>      GitHubGroup.create_group_repo (repo_name:str,
>                                     collaborators:[<class'str'>],
>                                     permission:str, rename_files={},
>                                     repo_template='', private=True,
>                                     description='', team_slug='',
>                                     team_permission='')

Create a Group Repository

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| repo_name | str |  | group repository name |
| collaborators | [<class 'str'>] |  | list of collaborators GitHub id |
| permission | str |  | the permission of collaborators. `pull`, `push` or `admin` |
| rename_files | dict | {} | dictionary of files renames {<og_name>:<new_name>} |
| repo_template | str |  | If empty string, an empty repo will be created. Put in the format of "<owner>/<repo>" |
| private | bool | True | visibility of the created repository |
| description | str |  | description for the GitHub repository |
| team_slug | str |  | team slug, add to this repo |
| team_permission | str |  | team permission to this repository `pull`, `push` or `admin` |
| **Returns** | **Repository** |  | **created repository** |

In [ ]:
repo = g.create_group_repo(
    repo_name="API_test_repo",
    collaborators=["jasongfleischer"],
    permission="admin",
    repo_template="COGS118A/group_template",
    rename_files={
        "Checkpoint_groupXXX.ipynb": "Checkpoint_group001.ipynb",
        "FinalProject_groupXXX.ipynb": "FinalProject_group001.ipynb",
        "Proposal_groupXXX.ipynb": "Proposal_group001.ipynb"
    },
    private=False,
    description="Test Creation of Group Repo for COGS118A final project group",
    team_slug="Instructors_Sp23",
    team_permission="admin"
)

Repo  API_test_repo  Created... Wait for 3 sec to updates
File Successfully Renamed from   Checkpoint_groupXXX.ipynb   to  Checkpoint_group001.ipynb 
File Successfully Renamed from   FinalProject_groupXXX.ipynb   to  FinalProject_group001.ipynb 
File Successfully Renamed from   Proposal_groupXXX.ipynb   to  Proposal_group001.ipynb 
Added Collaborator:  jasongfleischer  to:  API_test_repo  with permission:  admin 
Team  Instructors_Sp23  added to  API_test_repo  with permission  admin 
Group Repo:  API_test_repo  successfuly created!


In [ ]:
#| hide
# take down the repo that we just created
repo.delete()

# Lower Level Methods

Belows are the components that faciliate the `GitHubGroup.create_group_repo`. If errors were prompted during group creation scripts, or simply you want more flexibility, you can call those components individually.

## GitHub Repository Creation

_Note:_ `GtiHubGroup.create_group_repo` call this method to create a GitHub repository

`personal_account` argument controls the location of the repository creation. If set to `False` (default), it will create repository in the target organization. If set to `True`, the new repository will be created in the personal GitHub account.

In [ ]:
show_doc(GitHubGroup.create_repo)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/gh_group.py#L213){target="_blank" style="float:right; font-size:smaller"}

### GitHubGroup.create_repo

>      GitHubGroup.create_repo (repo_name:str, repo_template='', private=True,
>                               description='', personal_account=False)

Create a repository, either blank, or from a template

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| repo_name | str |  | repository name |
| repo_template | str |  | template repository that new repo will use. If empty string, an empty repo will be created. Put in the format of "<owner>/<repo>" |
| private | bool | True | visibility of the created repository |
| description | str |  | description for the GitHub repository |
| personal_account | bool | False | create repos in personal GitHub account |
| **Returns** | **Repository** |  |  |

_Note:_ `GtiHubGroup.create_group_repo` call this method to create a GitHub repository

`personal_account` argument controls the location of the repository creation. If set to `False` (default), it will create repository in the target organization. If set to `True`, the new repository will be created in the personal GitHub account.

In [ ]:
# create a repo under the org
repo = g.create_repo(
    "test-repo-organizational",
    private=True
)
print(repo)

repo.delete() #| hide_line

Repository(full_name="COGS118A/test-repo-organizational")


As you can see from the full name `COGS118A/test-repo`, it is created under the organization of `COGS118A`.

Alternatively, I can also create a new repository under my personal account.

In [ ]:
# create a repo under the my personal account
repo = g.create_repo(
    "test-repo-personal",
    private=True,
    personal_account=True
)
print(repo)

repo.delete() #| hide_line

Repository(full_name="scott-yj-yang/test-repo-personal")


In [ ]:
# create a repo under the my personal account
repo = g.create_repo(
    repo_name="test-repo-personal",
    private=True,
    personal_account=True
)
print(repo)

repo.delete() #| hide_line

Repository(full_name="scott-yj-yang/test-repo-personal")


### Create Repository from Template

You can also create a repository with a template repository. To do that, specify the full name of the template repository to the `repo_template` parameter. From the output, we can see that the repository `test-repo-from-template` is created with the template files. 

In [ ]:
# create a repo from a template
repo = g.create_repo(
    repo_name="test-repo-from-template",
    repo_template="COGS118A/group_template",
    private=True
)
print(repo)

# wait 3 sec for repository creation.
time.sleep(3)

print("\nThis Repository contains... \n")
pprint(repo.get_contents("."))

Repository(full_name="COGS118A/test-repo-from-template")

This Repository contains... 

[ContentFile(path=".gitignore"),
 ContentFile(path="Checkpoint_groupXXX.ipynb"),
 ContentFile(path="FinalProject_groupXXX.ipynb"),
 ContentFile(path="Proposal_groupXXX.ipynb"),
 ContentFile(path="README.md")]


## Rename Files in the Repository

Usually, the template file names are generics and is not specific to a group. Under the context of group project, we want to rename each notebook files with thier group numbers. For example, for Group 1, we want to rename the file `Checkpoint_groupXXX.ipynb` to `Checkpoint_group001.ipynb`. To do that, we can use the following method.

In [ ]:
show_doc(GitHubGroup.rename_files)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/gh_group.py#L262){target="_blank" style="float:right; font-size:smaller"}

### GitHubGroup.rename_files

>      GitHubGroup.rename_files (repo:github.Repository.Repository,
>                                og_filename:str, new_filename:str)

Rename the file by delete the old file and commit the new file

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| repo | Repository | the repository that we want to rename file |
| og_filename | str | old file name |
| new_filename | str | new file name |

_Note:_ This method simply delete the old files and create new files with the updated file name. Therefore, 2 commits for each file is expected. (1 for delete, 1 for re-upload). For example, if I want to rename 5 files, I will have 10 commits need to do in total.

In [ ]:
g.rename_files(
    repo=repo,
    og_filename="Checkpoint_groupXXX.ipynb",
    new_filename="Checkpoint_group001.ipynb"
)
# take a look at new files
print("\nThis Repository contains... \n")
pprint(repo.get_contents("."))

File Successfully Renamed from   Checkpoint_groupXXX.ipynb   to  Checkpoint_group001.ipynb 

This Repository contains... 

[ContentFile(path=".gitignore"),
 ContentFile(path="Checkpoint_group001.ipynb"),
 ContentFile(path="FinalProject_groupXXX.ipynb"),
 ContentFile(path="Proposal_groupXXX.ipynb"),
 ContentFile(path="README.md")]


Notice that the files were renamed as expected.

## Add Collaborators and Teams to Repository

Once the repository was created, we need to give the student team members proper permission to write to the repository and instructional team to be the admin of the repository. Those two functionalities are achieve by the following two methods.

In [ ]:
show_doc(GitHubGroup.add_collaborator)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/gh_group.py#L276){target="_blank" style="float:right; font-size:smaller"}

### GitHubGroup.add_collaborator

>      GitHubGroup.add_collaborator (repo:github.Repository.Repository,
>                                    collaborator:str, permission:str)

Add collaborator to the repository with specified permission

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| repo | Repository | target repository |
| collaborator | str | GitHub username of the collaborator |
| permission | str | `pull`, `push` or `admin` |

In [ ]:
# add collaborator to the repository with push permission
g.add_collaborator(
    repo=repo,
    collaborator="Andrina-iris",
    permission="push"
)

Added Collaborator:  Andrina-iris  to:  test-repo-from-template  with permission:  push 


In [ ]:
list(repo.get_pending_invitations())

[Invitation(id=225810031)]

In [ ]:
collaborators = list(repo.get_collaborators())
permission = [repo.get_collaborator_permission(i) for i in collaborators]
# look at the permission.
pprint({collaborator.login: perm for collaborator, perm in zip(collaborators, permission)})

{'jasongfleischer': 'admin', 'scott-yj-yang': 'admin'}


Additionally, course staffs should be in a team in the GitHub Organization in order to manage student repositories.

In [ ]:
show_doc(GitHubGroup.get_team)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/gh_group.py#L254){target="_blank" style="float:right; font-size:smaller"}

### GitHubGroup.get_team

>      GitHubGroup.get_team (team_slug:str)

Get the team inside the target organization

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| team_slug | str | team slug of the team |
| **Returns** | **Team** |  |

In [ ]:
team = g.get_team("Instructors_Sp23")
team

Team(name="Instructors_Sp23", id=7676644)

In [ ]:
show_doc(GitHubGroup.add_team)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/gh_group.py#L301){target="_blank" style="float:right; font-size:smaller"}

### GitHubGroup.add_team

>      GitHubGroup.add_team (repo:github.Repository.Repository,
>                            team:github.Team.Team, permission:str)

Add team to the repository with specified permission

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| repo | Repository | target repository |
| team | Team | GitHub username of the collaborator |
| permission | str | `pull`, `push` or `admin` |

In [ ]:
g.add_team(
    repo=repo,
    team=team,
    permission="admin"
)

Team  Instructors_Sp23  added to  test-repo-from-template  with permission  admin 


In [ ]:
collaborators = list(repo.get_collaborators())
permission = [repo.get_collaborator_permission(i) for i in collaborators]
# look at the permission.
pprint({collaborator.login: perm for collaborator, perm in zip(collaborators, permission)})

{'Dongze-Li': 'admin',
 'KevinZ0217': 'admin',
 'eleeeysh': 'admin',
 'jasongfleischer': 'admin',
 'scott-yj-yang': 'admin',
 'stephenjarrell19': 'admin',
 'vikizzz': 'admin'}


In [ ]:
#| hide
repo.delete()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()